# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [13]:
# imports

import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [14]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

Python-dotenv could not parse statement starting at line 8
Python-dotenv could not parse statement starting at line 10


# Import Data

In [15]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
neighborhoods_path = Path("neighborhoods_coordinates.csv")
neighborhoods_coordinates = pd.read_csv(neighborhoods_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [16]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""    
    housing_units = sfo_data.groupby('year')['housing_units'].mean()
    plt.rc('axes',titlesize=14, labelsize=12, titleweight='bold')
    
    return pn.pane.Matpoltlib(housing_units_per_year)

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    average_rent_sqr_foot = sfo_data["gross_rent"].groupby([sf_data.index]).mean()

    gross_rent_fig = plt.figure()
    average_rent_plot = (average_rent_sqr_foot.plot.line(x='year', y='gross_rent', title="Average Goss Rent per Year"))
    plt.close(gross_rent_fig)
    
    return pn.pane.Matplotlib(gross_rent_fig)

def average_sales_price():
    """Average Sales Price Per Year."""    
    average_price_sqr_foot = sfo_data["sale_price_sqr_foot"].groupby([sf_data.index]).mean()
    sales_price_fig = plt.figure()
    average_price_plot = (average_price_sqr_foot.plot.line(x='year', y='sale_price_sqr_foot', title="Average Sales Price per Year"))
    plt.close(sales_price_fig)
    
    return pn.pane.Matplotlib(sales_price_fig)


In [17]:
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""    
    sfo_data_new = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    sfo_sales = sfo_data_new["sale_price_sqr_foot"]
    sfo_sales_df = pd.DataFrame(sfo_sales).reset_index()
     
    
    return sfo_sales_df.loc[sfo_sales_df['neighborhood']==neighborhood,:].hvplot.line(
        x="year",
        y="sale_price_sqr_foot",
        colormap="coolwarm",
        title="SF Sale per Square foot per Year",
    )

sfo_data_new = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
sfo_sales = sfo_data_new["sale_price_sqr_foot"]
sfo_sales_df = pd.DataFrame(sfo_sales).reset_index()
neighborhood_choice = sfo_sales_df["neighborhood"].unique()

#interact(average_price_by_neighborhood, neighborhood=neighborhood_choice)


In [18]:
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""    
    avg_value_per_neighborhood = sfo_data.groupby([sfo_data["neighborhood"]]).mean()
    avg_value_per_neighborhood = avg_value_per_neighborhood.reset_index()
    top_10_most_expensive = avg_value_per_neighborhood.nlargest(10, 'sale_price_sqr_foot').reset_index()
    top10_plot = px.bar(top_10_most_expensive,
    x="neighborhood", 
    y="sale_price_sqr_foot", 
    title="Top 10 Most Expensive neighborhoods in San Francisco", 
    )

    return top10_plot

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    avg_values_neighborhood = neighborhood_df.drop(columns=['year']).groupby('neighborhood').mean().round(2)
    expensive_neighborhood = avg_values_neighborhood.sort_values("sale_price_sqr_foot", ascending = False).head(10)
  
    return expensive_neighborhood.hvplot.bar(
    
        x="neighborhood", 
        y="sale_price_sqr_foot", 
        title="Top 10 Most Expensive Neighborhoods in San Francisco", 
        height=400,
        rot=46)


In [19]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    avg_value_per_neighborhood = sfo_data.groupby([sf_data["neighborhood"]]).mean()    
    top_10_most_expensive = avg_value_per_neighborhood.nlargest(10, 'sale_price_sqr_foot').reset_index()
    top_10_parallel = px.parallel_coordinates(
    top_10_most_expensive, 
    color='sale_price_sqr_foot')

    return top_10_parallel


def parallel_categories():
    """Parallel Categories Plot."""    
    avg_value_per_neighborhood = sfo_data.groupby([sf_data["neighborhood"]]).mean()  
    top_10_ex_neighborhood = avg_value_per_neighborhood.nlargest(10, 'sale_price_sqr_foot').reset_index()
    top_10_parallel_categories = px.parallel_categories(
    top_10_ex_neighborhood,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
)

    return top_10_parallel_categories


def neighborhood_map():
    """Neighborhood Map"""  
    file_path = Path("Data/neighborhoods_coordinates.csv")
    df_neighborhood_location = pd.read_csv(file_path)
    avg_value_per_neighborhood = sfo_data.groupby([sf_data["neighborhood"]]).mean()  
    avg_value_per_neighborhood = avg_value_per_neighborhood.reset_index()
    avg_value_location = pd.concat([avg_value_per_neighborhood, df_neighborhood_location], axis="columns", join="inner")

    location_map = px.scatter_mapbox(
    avg_value_location,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    color_continuous_scale=px.colors.cyclical.IceFire,
    #title="Average Sale Price per Square Foot and Gross Rent in San Francisco",
    zoom=11,
    width=800,
)
    

    return location_map


In [20]:
df = px.data.gapminder().query("year == 2007")
fig = px.sunburst(df, path=['neighborhood', 'county'], values='pop',
    color='sale_price_sqr_foot',
    color_continuous_scale='RdBu',
    color_continuous_midpoint=np.average(df['sale_price_sqr_foot'], weights=df['pop']))
fig.show()

NameError: name 'np' is not defined

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard

title = '#Real Estate Analysis of San Francisco from 2010-2016'
welcome_tab = pn.Row(pn.Column(title), neighborhoods_map())

market_analysis_row = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())


neighborhoods_analysis_tab = pn.Column(interact(average_price_by_neighborhood, neighborhood=neighborhood_choice),
    top_most_expensive_neighborhoods()
)

parallel_plots_tab = pn.Column( 
    parallel_categories(),
    parallel_coordinates()
)


# Create a tab layout for the dashboard
# YOUR CODE HERE!
SF_dashboard = pn.Tabs(("Welcome", welcome_tab), 
("Yearly Market Analysis", market_analysis_row),
("Neighborhood Analysis", neighborhood_analysis_tab), 
("Interactive Market Analysis by Neighborhood", parallel_plots_tab))
# Create the dashboard

In [ ]:
# Serve the dashboard:
dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
housing_units_per_year()

In [ ]:
average_gross_rent()

In [ ]:
average_sales_price()

In [ ]:
average_price_by_neighborhood()

In [ ]:
top_most_expensive_neighborhoods()

In [ ]:
most_expensive_neighborhoods_rent_sales()

In [ ]:
neighborhood_map().show()

In [ ]:
parallel_categories()

In [ ]:
parallel_coordinates()

In [ ]:
sunburst()